# PySpark

In [1]:
# импорты
from pyspark.sql import SparkSession

from pyspark.sql import functions as F # типо Functions
from pyspark.sql import types as T # типо Types

from pyspark.sql.window import Window
from pyspark.sql.types import IntegerType, FloatType

## Основные операции

В данном блокноте будем работать с датасетом:
* Titanic Huge Dataset - 1M Passengersc: https://www.kaggle.com/datasets/marcpaulo/titanic-huge-dataset-1m-passengers

скачиваем файл `huge_1M_titanic.csv` и кладем в удобное место

In [2]:
# Инициализация SparkSession
spark = SparkSession.builder.getOrCreate()
spark

### Создание DataFrame для источника данных

В pandas мы бы просто считали **.csv** файл как DataFrame и работали с ним.

Но тут определяется DataFrame для источника данных, источником является **.csv** файл `huge_1M_titanic.csv`

In [3]:
df = spark.read.csv('./data/huge_1M_titanic.csv', header=True, inferSchema=False) 

# inferSchema=True - прочитать файл целиком, чтобы определить типы колонок 
# файлы могут быть очень большие 

In [ ]:
# При чтении определении DataFrame к источнику можно задать схему самостоятельно
# Пример:
# schema = T.StructType([
#     T.StructField("PassengerId", T.IntegerType(), True),
#     T.StructField("Name", T.StringType(), True),
# ])

# df = spark.read.csv("file.csv", header=True, schema=schema)

In [4]:
# Посмотреть план источника данных
# схема, путь до данных и другие характеристики 
df.explain()

== Physical Plan ==
FileScan csv [PassengerId#17,Survived#18,Pclass#19,Name#20,Sex#21,Age#22,SibSp#23,Parch#24,Ticket#25,Fare#26,Cabin#27,Embarked#28] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/c:/dev/mlRes/BigDataTech/seminars/sem_1/data/huge_1M_titanic.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<PassengerId:string,Survived:string,Pclass:string,Name:string,Sex:string,Age:string,SibSp:s...




In [5]:
# Кол-во партиций (фрагментов изначальных данных)
# Каждая партиция будет обрабатываться отдельным ядром 


# В данном примере файл `huge_1M_titanic.csv` разбит на 12 партиций (фрагментов)
df.rdd.getNumPartitions()

12

In [6]:
# Определить количество строк к df
print(df.count())

# Определить колонки df
print(df.columns)

# Определить схему df
df.printSchema()

1000000
['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']
root
 |-- PassengerId: string (nullable = true)
 |-- Survived: string (nullable = true)
 |-- Pclass: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- SibSp: string (nullable = true)
 |-- Parch: string (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: string (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



### Функция `.show()`

In [7]:
# Вывести первые 5 строк df
df.show(5)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+------------------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|              Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+------------------+-----+--------+
|       1310|       1|     1|Name1310, Miss. S...|female|NULL|    0|    0|SOTON/O2 3101272| 76.76016504643573| NULL|       C|
|       1311|       0|     3|Name1311, Col. Su...|  male|29.0|    0|    0|          223596|10.193096706320182| NULL|       S|
|       1312|       0|     3|Name1312, Mr. Sur...|  male|20.0|    0|    0|           54636| 12.02941641147422|  C83|       C|
|       1313|       0|     3|Name1313, Mr. Sur...|  male|27.0|    0|    0|        PC 17760|13.429447862759872| NULL|       S|
|       1314|       0|     3|Name1314, Mr. Sur...|  male|32.0|    0|    0|          364512| 4.840769450167068|  E33|  

In [87]:
# Позволяет делать вертикальный вывод, если строки слишком большие
df.show(5, vertical=True)

-RECORD 0---------------------------
 PassengerId | 1310                 
 Survived    | 1                    
 Pclass      | 1                    
 Name        | Name1310, Miss. S... 
 Sex         | female               
 Age         | NULL                 
 SibSp       | 0                    
 Parch       | 0                    
 Ticket      | SOTON/O2 3101272     
 Fare        | 76.76016504643573    
 Cabin       | NULL                 
 Embarked    | C                    
-RECORD 1---------------------------
 PassengerId | 1311                 
 Survived    | 0                    
 Pclass      | 3                    
 Name        | Name1311, Col. Su... 
 Sex         | male                 
 Age         | 29.0                 
 SibSp       | 0                    
 Parch       | 0                    
 Ticket      | 223596               
 Fare        | 10.193096706320182   
 Cabin       | NULL                 
 Embarked    | S                    
-RECORD 2---------------------------
 

### Функция `.select()`

In [8]:
# Вызывается у набора данных, в нашем случае у df

# Получить все столбцы
df.select("*").show(5)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+------------------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|              Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+------------------+-----+--------+
|       1310|       1|     1|Name1310, Miss. S...|female|NULL|    0|    0|SOTON/O2 3101272| 76.76016504643573| NULL|       C|
|       1311|       0|     3|Name1311, Col. Su...|  male|29.0|    0|    0|          223596|10.193096706320182| NULL|       S|
|       1312|       0|     3|Name1312, Mr. Sur...|  male|20.0|    0|    0|           54636| 12.02941641147422|  C83|       C|
|       1313|       0|     3|Name1313, Mr. Sur...|  male|27.0|    0|    0|        PC 17760|13.429447862759872| NULL|       S|
|       1314|       0|     3|Name1314, Mr. Sur...|  male|32.0|    0|    0|          364512| 4.840769450167068|  E33|  

Если не вызывать `show()`, то будет просто построен план вычисления, но оно не будет выполнено

In [13]:
df.select(df.Age) # То есть на данном этапе DataFrame, как результата выполнения df.select(df.Age) еще нет, есть только план для его вычисления

DataFrame[Age: double]

Когда будет вызвана любая из функций: `.count()`, `.collect()`, `.show()`, функции агрегации - будет получен DataFrame

In [9]:
# Передавать конкретные имена колонок
df.select("Age", "Survived", "Pclass").show(5)

+----+--------+------+
| Age|Survived|Pclass|
+----+--------+------+
|NULL|       1|     1|
|29.0|       0|     3|
|20.0|       0|     3|
|27.0|       0|     3|
|32.0|       0|     3|
+----+--------+------+
only showing top 5 rows



In [26]:
# Обращаться к колонкам можно как к полю объекта (через точку): df.Age
print(df.Age)

df.select(df.Age, df.Pclass).show(5)

Column<'Age'>
+----+------+
| Age|Pclass|
+----+------+
|NULL|     1|
|29.0|     3|
|20.0|     3|
|27.0|     3|
|32.0|     3|
+----+------+
only showing top 5 rows



In [ ]:
df.select(F.col('Age')).show(5)

+----+
| Age|
+----+
|NULL|
|29.0|
|20.0|
|27.0|
|32.0|
+----+
only showing top 5 rows



In [ ]:
# Можно сразу применять различные трансформации к колонкам внутри .select()
df.select(df.Name, F.upper(df.Name), df.Age, df.Age + 10).show(5)

+--------------------+--------------------+----+----------+
|                Name|         upper(Name)| Age|(Age + 10)|
+--------------------+--------------------+----+----------+
|Name1310, Miss. S...|NAME1310, MISS. S...|NULL|      NULL|
|Name1311, Col. Su...|NAME1311, COL. SU...|29.0|      39.0|
|Name1312, Mr. Sur...|NAME1312, MR. SUR...|20.0|      30.0|
|Name1313, Mr. Sur...|NAME1313, MR. SUR...|27.0|      37.0|
|Name1314, Mr. Sur...|NAME1314, MR. SUR...|32.0|      42.0|
+--------------------+--------------------+----+----------+
only showing top 5 rows



In [29]:
df.select("*", df.Name).show(5) # Можно заметить что колонка Name присутствует 2 раза

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+------------------+-----+--------+--------------------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|              Fare|Cabin|Embarked|                Name|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+------------------+-----+--------+--------------------+
|       1310|       1|     1|Name1310, Miss. S...|female|NULL|    0|    0|SOTON/O2 3101272| 76.76016504643573| NULL|       C|Name1310, Miss. S...|
|       1311|       0|     3|Name1311, Col. Su...|  male|29.0|    0|    0|          223596|10.193096706320182| NULL|       S|Name1311, Col. Su...|
|       1312|       0|     3|Name1312, Mr. Sur...|  male|20.0|    0|    0|           54636| 12.02941641147422|  C83|       C|Name1312, Mr. Sur...|
|       1313|       0|     3|Name1313, Mr. Sur...|  male|27.0|    0|    0|        PC 17760|13.429447862759872| NULL|  

In [ ]:
# Передавать множество выражений, например:
df.select(df.Age > 25, F.isnull(df.Age), F.isnotnull(df.Age), df.Age.between(25, 30)).show(10)

# Можно заметить, что это не выборка по нескольким условиям, это выбор всего, но к каждой строке применяются все переданные выражения,
# получаем DataFrame с bool значениями

+----------+-------------+-----------------+-----------------------------+
|(Age > 25)|(Age IS NULL)|(Age IS NOT NULL)|((Age >= 25) AND (Age <= 30))|
+----------+-------------+-----------------+-----------------------------+
|      NULL|         true|            false|                         NULL|
|      true|        false|             true|                         true|
|     false|        false|             true|                        false|
|      true|        false|             true|                         true|
|      true|        false|             true|                        false|
|     false|        false|             true|                        false|
|      NULL|         true|            false|                         NULL|
|      true|        false|             true|                         true|
|      NULL|         true|            false|                         NULL|
|      true|        false|             true|                        false|
+----------+-------------

### Функция `.filter()` and `.where()`

Для выбора строк по условию необходимо использовать функцию `.filter()`

`.where()` является алиасом для `.filter()`

`.filter()` также как и `.select()` строит план вычисления, DF получается только после вызова соответствующих функций 

In [38]:
df_filter_one = df.filter(df.Survived == 1)
df_filter_one

DataFrame[PassengerId: int, Survived: int, Pclass: int, Name: string, Sex: string, Age: double, SibSp: int, Parch: int, Ticket: string, Fare: double, Cabin: string, Embarked: string]

In [39]:
df_filter_one.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+------------------+-----------+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|              Fare|      Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+------------------+-----------+--------+
|       1310|       1|     1|Name1310, Miss. S...|female|NULL|    0|    0|SOTON/O2 3101272| 76.76016504643573|       NULL|       C|
|       1315|       1|     3|Name1315, Master....|female| 0.0|    0|    0|           29750|14.805817148933464|       NULL|       S|
|       1317|       1|     2|Name1317, Miss. S...|female|28.0|    0|    1|          113781|27.950047997073444|       NULL|       S|
|       1320|       1|     3|Name1320, Miss. S...|female|23.0|    1|    0|            7534|20.822545765670117|       NULL|       S|
|       1322|       1|     3|Name1322, Miss. S...|female|NULL|    1|    0|  

In [44]:
# Получается, что когда мы обращаемся к df_filter_one для вызова какой-либо функции, по построенному плану вычислений собирается DataFrame
print(df_filter_one.count())

df_filter_one.show(2)

381681
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+------------------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|              Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+------------------+-----+--------+
|       1310|       1|     1|Name1310, Miss. S...|female|NULL|    0|    0|SOTON/O2 3101272| 76.76016504643573| NULL|       C|
|       1315|       1|     3|Name1315, Master....|female| 0.0|    0|    0|           29750|14.805817148933464| NULL|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+------------------+-----+--------+
only showing top 2 rows



In [ ]:
# Если мы примерно оцениваем объем данных и понимаем, что они смогут поместиться к нам в память драйвера, можно конвертировать в pandas DataFrame и работать с ним

# Теперь в result_pdf будет находится DataFrame pandas
result_pdf = df_filter_one.toPandas()

In [81]:
del result_pdf

In [59]:
# Можно заметить разницу в Wall time для выполнения функции count() у pandas и PySpark
print("Pandas time:")
%time result_pdf.count()

print("\nPySpark time:")
%time df_filter_one.count()

Pandas time:
CPU times: total: 62.5 ms
Wall time: 62.9 ms

PySpark time:
CPU times: total: 0 ns
Wall time: 300 ms


381681

In [64]:
# Также можно выполнять фильтрацию по нескольким условиям
# "&" - и, "|" - или
df_filter_two = df.filter((df.Age > 30) & (df.Survived == 1))
df_filter_two.show(5)

+-----------+--------+------+--------------------+------+----+-----+-----+----------+-----------------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|    Ticket|             Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------+-----------------+-----+--------+
|       1323|       1|     1|Name1323, Mr. Sur...|  male|52.0|    0|    0|A.5. 18509|36.84335154107275|  E25|       S|
|       1362|       1|     1|Name1362, Mr. Sur...|  male|56.0|    1|    0|    382652|50.08934734350119| NULL|       S|
|       1376|       1|     2|Name1376, Mrs. Su...|female|36.0|    0|    0|    347088|16.68499978990871| NULL|       S|
|       1384|       1|     1|Name1384, Mrs. Su...|female|35.0|    0|    1|    226875|              0.0| NULL|       C|
|       1389|       1|     1|Name1389, Mr. Sur...|  male|33.0|    0|    0|C.A. 29178|33.66723594458008|  A10|       S|
+-----------+--------+------+-------------------

`.where()` выполняется аналогично `.filter()`

### Объединение выбора и фильтрации

In [72]:
# Например, мы хотим сделать выбор строк по условию
df.filter(df.Age == 29).select(df.Age, "Name").show(5)

+----+--------------------+
| Age|                Name|
+----+--------------------+
|29.0|Name1311, Col. Su...|
|29.0|Name1325, Mr. Sur...|
|29.0|Name1469, Miss. S...|
|29.0|Name1555, Mrs. Su...|
|29.0|Name1556, Mr. Sur...|
+----+--------------------+
only showing top 5 rows



### Еще функции `.collect()`, `.take()`, `.toPandas()`

* `.collect()` - получает список строк
* `.take()` - получает заданное число строк
* `.toPandas()` - формирует pandas DataFrame от полученных данных

In [83]:
# Строим план вычислений
filter_df_one = df.filter(df.Survived == 0)

%time result_rows = filter_df_one.collect()

CPU times: total: 3.19 s
Wall time: 6.39 s


In [84]:
print(len(result_rows))
print(result_rows[0])

del result_rows

618319
Row(PassengerId=1311, Survived=0, Pclass=3, Name='Name1311, Col. Surname1311', Sex='male', Age=29.0, SibSp=0, Parch=0, Ticket='223596', Fare=10.193096706320182, Cabin=None, Embarked='S')


In [85]:
result_rows_2 = filter_df_one.take(10)
print(len(result_rows_2))
result_rows_2[:2]

10


[Row(PassengerId=1311, Survived=0, Pclass=3, Name='Name1311, Col. Surname1311', Sex='male', Age=29.0, SibSp=0, Parch=0, Ticket='223596', Fare=10.193096706320182, Cabin=None, Embarked='S'),
 Row(PassengerId=1312, Survived=0, Pclass=3, Name='Name1312, Mr. Surname1312', Sex='male', Age=20.0, SibSp=0, Parch=0, Ticket='54636', Fare=12.02941641147422, Cabin='C83', Embarked='C')]

## Способы создания DataFrame'ов

Подробней прочитать можно: [тут](https://spark.apache.org/docs/latest/api/python/getting_started/quickstart_df.html#DataFrame-Creation)

### Добавление нового столбца

In [11]:
df.withColumn("FamilySize", df.Parch + df.SibSp).show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+------------------+-----+--------+----------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|              Fare|Cabin|Embarked|FamilySize|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+------------------+-----+--------+----------+
|       1310|       1|     1|Name1310, Miss. S...|female|NULL|    0|    0|SOTON/O2 3101272| 76.76016504643573| NULL|       C|         0|
|       1311|       0|     3|Name1311, Col. Su...|  male|29.0|    0|    0|          223596|10.193096706320182| NULL|       S|         0|
|       1312|       0|     3|Name1312, Mr. Sur...|  male|20.0|    0|    0|           54636| 12.02941641147422|  C83|       C|         0|
|       1313|       0|     3|Name1313, Mr. Sur...|  male|27.0|    0|    0|        PC 17760|13.429447862759872| NULL|       S|         0|
|       1314|       0|     3|Name1314, Mr

In [13]:
df.printSchema() # Можно заметить, что схема исходного источника данных осталась без изменений
# т.е df.withColumn("FamilySize", df.Parch + df.SibSp) также создает новый DataFrame

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [14]:
df = df.withColumn("FamilySize", df.Parch + df.SibSp)

In [15]:
df.printSchema() # Можно заметить появление FamilySize

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)
 |-- FamilySize: integer (nullable = true)



In [17]:
# Можно заметить, как стало стал отличаться физический план
df.explain()

== Physical Plan ==
*(1) Project [PassengerId#17, Survived#18, Pclass#19, Name#20, Sex#21, Age#22, SibSp#23, Parch#24, Ticket#25, Fare#26, Cabin#27, Embarked#28, (Parch#24 + SibSp#23) AS FamilySize#121]
+- FileScan csv [PassengerId#17,Survived#18,Pclass#19,Name#20,Sex#21,Age#22,SibSp#23,Parch#24,Ticket#25,Fare#26,Cabin#27,Embarked#28] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/c:/dev/mlRes/BigDataTech/seminars/sem_1/data/huge_1M_titanic.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<PassengerId:int,Survived:int,Pclass:int,Name:string,Sex:string,Age:double,SibSp:int,Parch:...




In [18]:
df.show(2)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+------------------+-----+--------+----------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|              Fare|Cabin|Embarked|FamilySize|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+------------------+-----+--------+----------+
|       1310|       1|     1|Name1310, Miss. S...|female|NULL|    0|    0|SOTON/O2 3101272| 76.76016504643573| NULL|       C|         0|
|       1311|       0|     3|Name1311, Col. Su...|  male|29.0|    0|    0|          223596|10.193096706320182| NULL|       S|         0|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+------------------+-----+--------+----------+
only showing top 2 rows



## Способы записи (сохранение DataFrame)

Позволяет записать полученный DataFrame в любой из поддерживаемых форматов файлов. Подробней: [тут](https://spark.apache.org/docs/latest/api/python/getting_started/quickstart_df.html#Getting-Data-In/Out)

## Группировка и агрегация

In [5]:
# Схоже с pandas, только отличе в том, что данные не хранятся все в памяти

# Определить количество людей в каждом из классов
result_gr_1 = df.groupby(df.Pclass).agg(
    F.count(df.PassengerId)
)

# Также строит план вычислений
print(result_gr_1)

DataFrame[Pclass: string, count(PassengerId): bigint]


In [7]:
# Вычислить
%time result_gr_1.show()

+------+------------------+
|Pclass|count(PassengerId)|
+------+------------------+
|     3|            553319|
|     1|            242069|
|     2|            204612|
+------+------------------+

CPU times: total: 15.6 ms
Wall time: 1.01 s


In [105]:
# Тоже строится план, а не сразу вычисляется
df.groupby(df.Pclass).count()

DataFrame[Pclass: int, count: bigint]

In [110]:
df.groupby(df.Pclass).count().show()
# df.groupby(df.Pclass).avg("PassengerId").show()

+------+------+
|Pclass| count|
+------+------+
|     1|242069|
|     3|553319|
|     2|204612|
+------+------+



In [112]:
# Можно выполнять группировку по нескольким столбцам

df.groupby(df.Pclass, df.Survived).count().show()

+------+--------+------+
|Pclass|Survived| count|
+------+--------+------+
|     1|       0| 90668|
|     3|       1|132666|
|     1|       1|151401|
|     2|       1| 97614|
|     2|       0|106998|
|     3|       0|420653|
+------+--------+------+



In [8]:
# Поддержка нескольких функций агрегации. Также можно установить псевдонимы
df.groupBy(df.Pclass, df.Survived).agg(
    F.avg(df.Age).alias("mean_age"),
    F.count(df.PassengerId).alias("psngr_count")
).show()

+------+--------+------------------+-----------+
|Pclass|Survived|          mean_age|psngr_count|
+------+--------+------------------+-----------+
|     1|       1| 34.14365493120401|     151401|
|     2|       0| 32.45753543095491|     106998|
|     3|       1|23.244919110926855|     132666|
|     3|       0|  27.0155267639002|     420653|
|     2|       1| 26.09276820107242|      97614|
|     1|       0|40.196781121788696|      90668|
+------+--------+------------------+-----------+



### Сортировка: `.orderBy()`

Одна из самых затратных операций

In [12]:
# Можно, например, отсортировать финальный результат, группировки или какой-либо выборки, фильтрации

# Без сортировки
%time df.filter(df.Survived==1).groupby(df.Pclass).agg(F.count(df.PassengerId).alias("passanger_count"), F.mean(df.Age).alias("mean_age")).show()

# С сортировкой
%time df.filter(df.Survived==1).groupby(df.Pclass).agg(F.count(df.PassengerId).alias("passanger_count"), F.mean(df.Age).alias("mean_age")).orderBy("passanger_count").show()

+------+---------------+------------------+
|Pclass|passanger_count|          mean_age|
+------+---------------+------------------+
|     1|         151401| 34.14365493120401|
|     2|          97614| 26.09276820107242|
|     3|         132666|23.244919110926855|
+------+---------------+------------------+

CPU times: total: 0 ns
Wall time: 465 ms
+------+---------------+------------------+
|Pclass|passanger_count|          mean_age|
+------+---------------+------------------+
|     2|          97614| 26.09276820107242|
|     3|         132666|23.244919110926855|
|     1|         151401| 34.14365493120401|
+------+---------------+------------------+

CPU times: total: 15.6 ms
Wall time: 446 ms


In [15]:
# Например, сколько времени было затрачено на выполнение сортировки
%time df.orderBy(df.Fare).show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------+----+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|    Ticket|Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------+----+-----+--------+
|     962962|       0|     2|Name962962, Mr. S...|  male|51.0|    0|    0|     13502| 0.0|  C78|       S|
|       1718|       1|     1|Name1718, Miss. S...|female|23.0|    0|    0|A.5. 18509| 0.0| C148|       C|
|     788890|       0|     2|Name788890, Mr. S...|  male|40.0|    0|    0|    113784| 0.0| NULL|       S|
|       2038|       0|     2|Name2038, Mr. Sur...|  male|27.0|    0|    0|     13568| 0.0|  C87|       S|
|     266498|       0|     3|Name266498, Miss....|female|15.0|    1|    0|    349909| 0.0| NULL|       S|
|       1719|       0|     3|Name1719, Mr. Sur...|  male|NULL|    0|    0|    234686| 0.0| NULL|       S|
|     788885|       0|     1|Name788885, Mr. S

In [20]:
%time df.sort(df.Fare).show()

+-----------+--------+------+--------------------+------+----+-----+-----+------------------+----+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|            Ticket|Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+------------------+----+-----+--------+
|     963134|       0|     2|Name963134, Mr. S...|  male|55.0|    0|    0|            347077| 0.0| NULL|       S|
|     440547|       1|     1|Name440547, Miss....|female|66.0|    0|    1|              3460| 0.0| NULL|       C|
|       1665|       1|     3|Name1665, Miss. S...|female|34.0|    1|    1|            349225| 0.0| NULL|       S|
|     440700|       1|     1|Name440700, Mr. S...|  male|27.0|    1|    0|SOTON/O.Q. 3101305| 0.0|  C49|       C|
|     527618|       0|     2|Name527618, Mr. S...|  male|34.0|    0|    0|            315093| 0.0| NULL|       S|
|     440515|       0|     1|Name440515, Mr. S...|  male|33.0|    0|    0|         A/5 2

## Оконные функции

be later ...

## SQL 

Spark поддерживает работу с SQL, можно писать запросы на привычном SQL, как при работе с базой данных

In [5]:
# Необходимо зарегистрировать df как таблицу (дать имя, например: tableOne)
df.createOrReplaceTempView("tableOne")

In [10]:
# Также строится план вычислений
calculate_plan_sql = spark.sql("SELECT * FROM tableOne")

In [ ]:
calculate_plan_sql.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+------------------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|              Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+------------------+-----+--------+
|       1310|       1|     1|Name1310, Miss. S...|female|NULL|    0|    0|SOTON/O2 3101272| 76.76016504643573| NULL|       C|
|       1311|       0|     3|Name1311, Col. Su...|  male|29.0|    0|    0|          223596|10.193096706320182| NULL|       S|
|       1312|       0|     3|Name1312, Mr. Sur...|  male|20.0|    0|    0|           54636| 12.02941641147422|  C83|       C|
|       1313|       0|     3|Name1313, Mr. Sur...|  male|27.0|    0|    0|        PC 17760|13.429447862759872| NULL|       S|
|       1314|       0|     3|Name1314, Mr. Sur...|  male|32.0|    0|    0|          364512| 4.840769450167068|  E33|  

: 

In [ ]:
spark.sql("SELECT Pclass, COUNT(PassengerId) as count_pssng FROM tableOne GROUP BY Pclass ORDER BY count_pssng")

Py4JJavaError: An error occurred while calling o23.sql.
: java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://wiki.apache.org/hadoop/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:735)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:270)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:286)
	at org.apache.hadoop.fs.RawLocalFileSystem.setPermission(RawLocalFileSystem.java:978)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkOneDirWithMode(RawLocalFileSystem.java:660)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:700)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:672)
	at org.apache.hadoop.fs.ChecksumFileSystem.mkdirs(ChecksumFileSystem.java:788)
	at org.apache.spark.sql.catalyst.catalog.InMemoryCatalog.liftedTree1$1(InMemoryCatalog.scala:122)

: 

In [8]:
spark.sql("SELECT PassengerId, Name, Fare FROM tableOne").show()

+-----------+--------------------+------------------+
|PassengerId|                Name|              Fare|
+-----------+--------------------+------------------+
|       1310|Name1310, Miss. S...| 76.76016504643573|
|       1311|Name1311, Col. Su...|10.193096706320182|
|       1312|Name1312, Mr. Sur...| 12.02941641147422|
|       1313|Name1313, Mr. Sur...|13.429447862759872|
|       1314|Name1314, Mr. Sur...| 4.840769450167068|
|       1315|Name1315, Master....|14.805817148933464|
|       1316|Name1316, Mr. Sur...|   115.80634419925|
|       1317|Name1317, Miss. S...|27.950047997073444|
|       1318|Name1318, Mr. Sur...|29.467202626662697|
|       1319|Name1319, Mr. Sur...|               0.0|
|       1320|Name1320, Miss. S...|20.822545765670117|
|       1321|Name1321, Mr. Sur...|  4.79782744017412|
|       1322|Name1322, Miss. S...|12.462809150380991|
|       1323|Name1323, Mr. Sur...| 36.84335154107275|
|       1324|Name1324, Mr. Sur...|               0.0|
|       1325|Name1325, Mr. S